In [ ]:
# default_exp annotation.yolo_adapter

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

In [ ]:
# export

import csv
import sys
import argparse
import logging
from os.path import join, splitext, basename, dirname, isfile
from mlcore.category_tools import read_categories
from mlcore.io.core import scan_files, create_folder
from mlcore.image.pillow_tools import get_image_size
from mlcore.annotation.core import Annotation, AnnotationAdapter, Region, RegionShape

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

FIELD_NAMES = ['class_number', 'c_x', 'c_y', 'width', 'height']
DEFAULT_CATEGORIES_FILE = 'categories.txt'

In [ ]:
# export

logger = logging.getLogger(__name__)

# YOLO Annotation Adapter
> YOLO annotation adapter.

This adapter is tested with the [YOLOv3 Pytorch](https://github.com/ultralytics/yolov3) and [YOLOv5 Pytorch](https://github.com/ultralytics/yolov5) repositories.
Bounding boxes are normalized between [0,1].
Images should be in a separate folder (named e.g. *images/*).
Annotations should be in a separate folder (named e.g. *labels/*) and must have the same file name as corresponding image source file with the ending *.txt*. Below is an example structure:

```
categories.txt
images/
    image1.jpg
labels/
    image1.txt
```

The `args` parameter contains the following options:
- `files_path`: the path to the folder containing the source files (e.g.: *data/image_object_detection/my_collection/trainval/images*)
- `annotations_path`: The path to the multi classification CSV annotation file (e.g.: data/image_object_detection/my_collection/trainval/labels)
- `categories_file`: The path to the categories file. If not explicitly set try to infer the file *categories.txt* from files_path (e.g.: data/image_object_detection/my_collection/categories.txt)

In [ ]:
# export


class YOLOAdapter(AnnotationAdapter):
    """
    Adapter to read and write annotations in the YOLO format.
    `args`: the arguments containing the parameters
    """

    def __init__(self, args):
        super().__init__()
        self.files_path = args.files_path
        self.annotations_path = args.annotations_path
        if args.categories_file is None:
            self.categories_file = join(dirname(dirname(self.files_path)), DEFAULT_CATEGORIES_FILE)
        else:
            self.categories_file = args.categories_file

    def read(self):
        """
        Reads YOLO annotations from the annotations folder.
        return: the annotations as dictionary
        """
        annotations = {}
        logger.info('Read annotations from {}'.format(self.annotations_path))

        annotation_files = scan_files(self.annotations_path, file_extensions='.txt')
        categories = read_categories(self.categories_file)
        categories_len = len(categories)
        skipped_annotations = []

        for annotation_file in annotation_files:
            with open(annotation_file, newline='') as csv_file:
                annotation_file_name = basename(annotation_file)
                file_name, _ = splitext(annotation_file_name)
                file_path = join(self.files_path, '{}{}'.format(file_name, '.jpg'))

                if not isfile(file_path):
                    logger.warning("{}: Source file not found, skip annotation.".format(file_path))
                    skipped_annotations.append(file_path)
                    continue

                if annotation_file not in annotations:
                    annotations[annotation_file] = Annotation(annotation_id=annotation_file, file_path=file_path)

                annotation = annotations[annotation_file]

                reader = csv.DictReader(csv_file, fieldnames=FIELD_NAMES, delimiter=' ')
                _, image_width, image_height = get_image_size(file_path)
                for row in reader:
                    c_x = float(row["c_x"])
                    c_y = float(row["c_y"])
                    width = float(row["width"])
                    height = float(row["height"])
                    class_number = int(row["class_number"])
                    # denormalize bounding box
                    x_min = _denormalize_value(c_x - (width / 2), image_width)
                    y_min = _denormalize_value(c_y - (height / 2), image_height)
                    x_max = x_min + _denormalize_value(width, image_width)
                    y_max = y_min + _denormalize_value(height, image_height)
                    points_x = [x_min, x_max]
                    points_y = [y_min, y_max]

                    labels = [categories[class_number]] if class_number < categories_len else []
                    if not labels:
                        logger.warning("{}: Class number exceeds categories, set label as empty.".format(
                            annotation_file
                        ))
                    region = Region(shape=RegionShape.RECTANGLE, points_x=points_x, points_y=points_y, labels=labels)
                    annotation.regions.append(region)

        logger.info('Finished read annotations')
        logger.info('Annotations read: {}'.format(len(annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))

        return annotations

    def write(self, annotations):
        """
        Writes YOLO annotations to the annotations folder and copy the corresponding source files.
        `annotations`: the annotations to write
        """
        annotations_folder = create_folder(self.annotations_path)
        target_folder = create_folder(self.files_path)
        categories = read_categories(self.categories_file)

        logger.info('Write annotations to {}'.format(annotations_folder))
        logger.info('Write file sources to {}'.format(target_folder))

        skipped_annotations = []

        for annotation in annotations.values():
            file_name, _ = splitext(basename(annotation.file_path))
            annotations_file = join(annotations_folder, '{}{}'.format(file_name, '.txt'))
            target_file = join(target_folder, basename(annotation.file_path))

            if not isfile(annotation.file_path):
                logger.warning("{}: Source file not found, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue
            if isfile(target_file):
                logger.warning("{}: Target file already exist, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue

            _, image_width, image_height = get_image_size(annotation.file_path)
            with open(annotations_file, 'w', newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=FIELD_NAMES, delimiter=' ')

                skipped_regions = []
                for index, region in enumerate(annotation.regions):
                    if region.shape != RegionShape.RECTANGLE:
                        logger.warning('Unsupported shape {}, skip region {} at path: {}'.format(region.shape,
                                                                                                 index,
                                                                                                 annotations_file))
                        skipped_regions.append(region)
                        continue

                    x_min, x_max = region.points_x
                    y_min, y_max = region.points_y
                    width = x_max - x_min
                    height = y_max - y_min
                    # normalize bounding box
                    c_x = _normalize_value(x_min + width / 2, image_width)
                    c_y = _normalize_value(y_min + height / 2, image_height)
                    width = _normalize_value(width, image_width)
                    height = _normalize_value(height, image_height)
                    label = region.labels[0] if len(region.labels) else ''
                    try:
                        class_number = categories.index(label)
                    except ValueError:
                        logger.warning('Unsupported label {}, skip region {} at path: {}'.format(label,
                                                                                                 index,
                                                                                                 annotations_file))
                        skipped_regions.append(region)
                        continue

                    writer.writerow(dict(zip(FIELD_NAMES, [class_number, c_x, c_y, width, height])))

                if len(skipped_regions) == len(annotation.regions):
                    logger.warning("{}: All regions skipped, skip annotation.".format(annotation.file_path))
                    skipped_annotations.append(annotation.file_path)

        logger.info('Finished write annotations')
        logger.info('Annotations written: {}'.format(len(annotations) - len(skipped_annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))

    @classmethod
    def argparse(cls, prefix=None):
        """
        Returns the argument parser containing argument definition for command line use.
        `prefix`: a parameter prefix to set, if needed
        return: the argument parser
        """
        parser = argparse.ArgumentParser()
        parser.add_argument(cls.assign_prefix('--files_path', prefix),
                            dest="files_path",
                            help="The path to the folder containing the files.",
                            required=True)
        parser.add_argument(cls.assign_prefix('--annotations_file', prefix),
                            dest="annotations_path",
                            help="The path to the folder containing the annotations.",
                            required=True)
        parser.add_argument(cls.assign_prefix('--categories_file', prefix),
                            dest="categories_file",
                            help="The path to the categories file. If not explicitly set try to infer from files_path.",
                            default=None)
        return parser

In [ ]:
show_doc(YOLOAdapter.read)
show_doc(YOLOAdapter.write)
show_doc(YOLOAdapter.argparse)

In [ ]:
# export


def _denormalize_value(value, metric):
    """
    Denormalize a bounding box value
    `value`: the value to denormalize
    `metric`: the metric to denormalize from
    return: the denormalized value
    """
    return int(value * metric)

In [ ]:
# export


def _normalize_value(value, metric):
    """
    Normalize a bounding box value
    `value`: the value to normalize
    `metric`: the metric to normalize against
    return: the normalized value
    """
    return float(value) / metric

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)


In [ ]:
# hide

# for generating scripts from notebook directly
notebook2script()

Converted annotation-core.ipynb.
Converted annotation-folder_category_adapter.ipynb.
Converted annotation-multi_category_adapter.ipynb.
Converted annotation-via_adapter.ipynb.
Converted annotation-viewer.ipynb.
Converted annotation-yolo_adapter.ipynb.
Converted category_tools.ipynb.
Converted core.ipynb.
Converted dataset-core.ipynb.
Converted dataset-image_classification.ipynb.
Converted dataset-image_object_detection.ipynb.
Converted dataset-image_segmentation.ipynb.
Converted dataset-type.ipynb.
Converted dataset_generator.ipynb.
Converted evaluation-core.ipynb.
Converted geometry.ipynb.
Converted image-color_palette.ipynb.
Converted image-inference.ipynb.
Converted image-opencv_tools.ipynb.
Converted image-pillow_tools.ipynb.
Converted image-tools.ipynb.
Converted index.ipynb.
Converted io-core.ipynb.
Converted tensorflow-tflite_converter.ipynb.
Converted tensorflow-tflite_metadata.ipynb.
Converted tensorflow-tfrecord_builder.ipynb.
Converted tools-check_double_images.ipynb.
Conver